In [2]:
import pandas as pd

In [6]:
details_df = pd.read_csv('../Data/Seleccion/details.csv')

In [7]:
details_df = details_df.set_index('EVENT_ID')

In [8]:
#Comprobamos que atributos tienen valores nulos
details_df.isnull().any()

EPISODE_ID            True
EVENT_TYPE           False
BEGIN_DATE_TIME      False
END_DATE_TIME        False
INJURIES_DIRECT      False
INJURIES_INDIRECT    False
DEATHS_DIRECT        False
DEATHS_INDIRECT      False
DAMAGE_PROPERTY       True
DAMAGE_CROPS          True
MAGNITUDE             True
BEGIN_LAT             True
BEGIN_LON             True
END_LAT               True
END_LON               True
dtype: bool

In [9]:
#Eliminamos aquellos eventos que no tengan asociado un episodio (tormenta)
details_df = details_df[details_df.EPISODE_ID.notnull()]

In [30]:
#Convertimos los episodios a tipo entero
details_df.loc[:, 'EPISODE_ID'] = details_df['EPISODE_ID'].apply(lambda x: int(x))

In [31]:
#Damos formato a las fechas
details_df.loc[:, 'BEGIN_DATE_TIME'] = pd.to_datetime(details_df['BEGIN_DATE_TIME'])
details_df.loc[:, 'END_DATE_TIME'] = pd.to_datetime(details_df['END_DATE_TIME'])

In [32]:
#Damos el valor por defecto de 0 al numero de muertes directas e indirectas, perdidas economicas generales y en cultivos,
# heridos directos e indirectos
details_df[['INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT', 'DAMAGE_CROPS',
            'DEATHS_INDIRECT', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS']] = details_df[['INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT', 'DAMAGE_CROPS',
            'DEATHS_INDIRECT', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS']].fillna('0')

In [33]:
#Convertismos las perdidas de propiedades y de cultivos en un numero entero transformando las distintas magnitudes
# en una comun ($)
def procesar_perdidas(x):
    if x == '0' or x == 0:
        return int(x)
    else:
        if x[-1] == 'K':
            mult = 1e3
        elif x[-1] == 'M':
            mult = 1e6
        else:
            mult = 1
        
        return int(float(x[:-1]) * mult)

In [34]:
details_df['DAMAGE_PROPERTY'] = details_df['DAMAGE_PROPERTY'].apply(procesar_perdidas)

In [35]:
details_df['DAMAGE_CROPS'] = details_df['DAMAGE_CROPS'].apply(procesar_perdidas)

In [36]:
# Observamos que los valores de las perdidas en muchos casos es la misma, lo cual indica que se introdujeron
# de forma poco precisa
details_df['DAMAGE_PROPERTY'].unique()

array([        0,     10000,      1000,     15000,     75000,      2000,
            5000,     20000,     25000,      8000,     17000,      9000,
            3000,      1500,       250,      4000,       500,      7000,
            6000,      1250,     12000,     21000,     18000,      7500,
             200,     50000,     90000,     30000,    100000,    500000,
           70000,   1000000,    147000,   1500000,    105000,     55000,
           45000,     35000,     43000,     20500,     80000,     40000,
          206000,    300000,    200000,    400000,     65000,     16000,
          150000,    110000,    250000,    350000,       800,     28000,
           60000,    750000,       100,    975000,      3500,       750,
          125000,     12500,   3500000,    120000,      9500,     13000,
            2500,     36000,      2250,     14000,    420000,   3000000,
         2000000,   6000000,  15000000,   8000000,   5000000,     85000,
             300,  10000000,    450000,  35000000, 

In [37]:
# Lo mismo ocurre con las perdidas economicas en cultibos
details_df.DAMAGE_CROPS.unique()

array([        0,     50000,     30000,     20000,     10000,       250,
            1000,       500,      3000,      5000,      2000,       100,
             200,   2000000,      6000,     25000,       300,       750,
            8000,      7000,      2500,     12000,      4000,    300000,
          200000,     40000,   3030000,    620000,    100000,   1000000,
            1500,     75000,    400000,    250000,     15000,  10000000,
          500000,  60000000,  30000000, 100000000,  45000000, 150000000,
        50000000,  90000000,  15000000,   5000000,   4000000,  70000000,
         8000000,   7000000,   3000000,   6000000])

In [38]:
# Quitamos los nulos de las magnitudes y los ponemos a 0
details_df['MAGNITUDE'] = details_df['MAGNITUDE'].fillna(0)

In [39]:
# Pasamos a preprocesar las posiciones iniciales y finales
import matplotlib.pyplot as plt

plt.scatter(details_df.BEGIN_LAT, details_df.BEGIN_LON)
plt.show()

plt.scatter(details_df.END_LAT, details_df.END_LON)
plt.show()

#Observamos que estan correctamente introducidos los que hay

In [48]:
#Hay demasiados nulos en todas las posiciones
details_df['END_LON'].isnull().sum()

6964

In [40]:
"""
Medidas a tomar:

1. Establecer como valores INI_LON y INI_LAT los de locations.
2. Establecer como valores protipicos de INI_LON e INI_LAT:
        if hay algun evento en el mismo episodio con estos valores: la media de ellos
        else 
            if ese evento tiene END_LON y END_LAT añadirlo
            elif algun evento de ese episodio tiene END_LON y END_LAT: la media de ellos
            else 
                eliminarlo

3. Establecer como valores END_LON y END_LAT los de details.csv
4. Establecer como valores protipicos de END_LON y END_LAT:
       if hay algun evento con el mismo episodio: la media de ellos
       else
           if ese evento tiene INI_LON y INI_LAT añadirlo
           elif algun evento tiene INI_LON y INI_LAT del mismo episodio: media de ellos
           else
               eliminarlo
"""

'\nMedidas a tomar:\n\n1. Establecer como valores INI_LON y INI_LAT los de locations.\n2. Establecer como valores protipicos de INI_LON e INI_LAT:\n        if hay algun evento en el mismo episodio con estos valores: la media de ellos\n        else \n            if ese evento tiene END_LON y END_LAT a\xc3\xb1adirlo\n            elif algun evento de ese episodio tiene END_LON y END_LAT: la media de ellos\n            else \n                eliminarlo\n\n3. Establecer como valores END_LON y END_LAT los de details.csv\n4. Establecer como valores protipicos de END_LON y END_LAT:\n       if hay algun evento con el mismo episodio: la media de ellos\n       else\n           if ese evento tiene INI_LON y INI_LAT a\xc3\xb1adirlo\n           elif algun evento tiene INI_LON y INI_LAT del mismo episodio: media de ellos\n           else\n               eliminarlo\n'

In [41]:
# Tenemos 5259 eventos que han provocado perdidas economicas de caracter general
details_df[details_df.DAMAGE_PROPERTY > 0].shape[0]

5259

In [42]:
# Tenemos 493 eventos que han provocado perdidas economicas en cultivos
details_df[details_df.DAMAGE_CROPS > 0].shape[0]

493

In [43]:
# Tenemos 164 eventos que han provocado heridos directamente
details_df[details_df.INJURIES_DIRECT > 0].shape[0]

164

In [44]:
# Tenemos 46 eventos que han provocado heridos indirectamente
details_df[details_df.INJURIES_INDIRECT > 0].shape[0]

46

In [45]:
# Tenemos 91 eventos que han provocado muertes directamente
details_df[details_df.DEATHS_DIRECT > 0].shape[0]

91

In [46]:
# Tenemos 46 eventos que han provocado muertes indirectamente
details_df[details_df.INJURIES_INDIRECT > 0].shape[0]

46

In [47]:
details_df.to_csv('../Data/Preproceso/details.csv')